In [5]:
from datetime import datetime
import pymongo
from pymongo import MongoClient

In [6]:
# from dbConnectionScript import connectionScript
import pandas as pd

env = "uat"

dbName = "assurekitDS"

collection = "companyScrapingData"

p = connectionScript(env = env, dbName = dbName, collectionName= collection)

In [7]:
data = list(p.collection.find({},{"_id":0}).sort("_id",1).limit(300000))
# df = pd.DataFrame(data)
# print(df.to_dict("records")[:1])
# df['id'] = [x for x in range(1196082,len(data))]

In [8]:
df = pd.DataFrame(data)
# print(df.to_dict("records")[:1])

In [9]:
df.rename(columns = {
    "0":"companyId",
    "2":"name",
    "4":"govStatus",
    # "class" : "LLP",
    # "subCategory" : "LLP",
    "16":"registrationDate",
    # "registrationState" : "Bangalore",
    # "authorisedCapital" : "",
    # "paidUpCapital" : "",
    "8":"business",
    "51":"address",
    "6":"registrationAuthority",
    "48":"emailAddress"
    # "dateOfIncorporation" : 2017,
    # "id" : "1196082",
    # "status" : "active"
},inplace = True)
print(df.to_dict("records")[:1])



[{'companyId': 'AAK-8285', '1': 'Company Name', 'name': 'CENTURION CONSULTING LLP', '3': 'Company Status', 'govStatus': 'Active', '5': 'RoC', 'registrationAuthority': 'RoC-Delhi', '7': 'Main division of business activity to be carried out in India ', 'business': 'Other Business Activities', '9': 'Description of main division', '10': 'Other Business Activities', '11': 'Number Of Partners ', '12': '0', '13': 'Number of Designated Partners', '14': '2', '15': 'Date of Incorporation', 'registrationDate': '11 October 2017', '17': 'Age of Company', '18': '5 years, 7 month, 6 days', '19': 'Login to view previous names', '20': 'Login to view previous cins', '21': 'Total Obligation of Contribution ', '22': '₹100,000', '23': 'Number of Employees', '24': 'Login to view', '25': 'Date of last financial year end date for which Statement of Accounts and Solvency filed', '26': 'N/A', '27': 'Date of last financial year end date for which Annual Return filed', '28': 'N/A', '29': 'Balance Sheet', '30': 'P

In [10]:
df = df[['companyId','name','govStatus',"registrationDate",'business','address','registrationAuthority','emailAddress']]


In [11]:
df = df[df["govStatus"] == "Active"]

In [12]:
df.drop_duplicates(subset = ['companyId'],inplace = True,keep = "first")

In [13]:
df.groupby('govStatus').count()

,companyId,name,registrationDate,business,address,registrationAuthority,emailAddress
govStatus,,,,,,,
Active,186340,186340,186340,186340,186340,186340,186340


In [14]:
email = "Email ID: pramjeet@gmail.com"

In [15]:
def cleanEmailId(email):
        articlesToRemove = len("Email ID: ")
        return email[articlesToRemove:] if "Email ID:" in email else email
    
def getPincode(address):
    return address[-6:]

def checkStringIsNumerics(pin):
    return True if pin.isnumeric() else False




In [16]:
df['emailAddress'] = df['emailAddress'].apply(cleanEmailId)

In [17]:
df['x'] = pd.to_datetime(df['registrationDate'],format = "%d %B %Y", errors = "coerce")

In [18]:
df = df[df['x'].notna()]

In [19]:
df['registrationDate'] = df['x']

In [20]:
df.drop(columns = ['x'], inplace = True)

In [21]:
df['pincode'] = df['address'].apply(getPincode)

In [22]:
df['checkPin'] = df['pincode'].apply(checkStringIsNumerics)

In [23]:
df['pin'] = df['pincode'].apply(lambda x:x if x.isnumeric() else "")

In [24]:
df.rename(columns = {"pin":"companyPincode"},inplace = True)

In [25]:
df.drop(columns = ['pincode',"checkPin"], inplace = True)

In [26]:
[x for x in range(5,10)]

[5, 6, 7, 8, 9]

In [27]:
len(df.index)

186261

In [24]:
#[x for x in range(1238133, 1238133 +len(df.index))]

In [25]:
# df['dateOfIncorporation'] = [x for x in range(1196082, len(df.index)]
#df['id'] = [x for x in range(1238133, 1238133 +len(df.index))]

In [28]:
df['dateOfIncorporation'] = df['registrationDate'].dt.year

In [29]:
df['registrationDate'] = df['registrationDate'].astype(str)

In [30]:
# df['dateOfIncorporation']
df['class'] = "LLP"
df['subCategory'] = "LLP"
df['registrationState'] = ""
df['authorisedCapital'] = ""
df['paidUpCapital'] = ""
df['status'] = "active"

In [31]:
#df['id'] = df['id'].astype(str)

In [32]:
# df[df['id'] == "1238133"]

In [33]:
df = df[df['dateOfIncorporation'] <= 2019]

In [29]:
df.to_dict("records")[:1]

[{'companyId': 'AAK-8285',
  'name': 'CENTURION CONSULTING LLP',
  'govStatus': 'Active',
  'registrationDate': '2017-10-11',
  'business': 'Other Business Activities',
  'address': 'Dabri Mod, RZ-74A, SITA PURI 1 GALI NO.9 NEW DELHI West Delhi DL 110059 IN',
  'registrationAuthority': 'RoC-Delhi',
  'emailAddress': ' pramjeet@gmail.com',
  'companyPincode': '',
  'dateOfIncorporation': 2017,
  'class': 'LLP',
  'subCategory': 'LLP',
  'registrationState': '',
  'authorisedCapital': '',
  'paidUpCapital': '',
  'status': 'active'}]

In [30]:
#df.to_excel("llpSet1().xlsx", index = False)

In [34]:
df['business'] = df['business'].apply(lambda x: x if x != "-" else "Other Business Activities")

In [35]:
df

,companyId,name,govStatus,registrationDate,business,address,registrationAuthority,emailAddress,companyPincode,dateOfIncorporation,class,subCategory,registrationState,authorisedCapital,paidUpCapital,status
0,AAK-8285,CENTURION CONSULTING LLP,Active,2017-10-11,Other Business Activities,"Dabri Mod, RZ-74A, SITA PURI 1 GALI NO.9 NEW D...",RoC-Delhi,pramjeet@gmail.com,,2017,LLP,LLP,,,,active
1,AAJ-4794,HAYA ONLINE SERVICES LLP,Active,2017-05-23,"Retail trade,except of motor vehicles and moto...","FL No B- 802, Meghvarsha S - 119/2/3, Warje, N...",RoC-Pune,kelkarnayna@gmail.com,,2017,LLP,LLP,,,,active
2,AAQ-3025,EKSYA EXPORTS LLP,Active,2019-08-20,Other Business Activities,"No. 93, Kavita, Ground floor, Seeta circle SBM...",RoC-Bangalore,anand.bv1991@gmail.com,,2019,LLP,LLP,,,,active
4,AAQ-8595,SUNNYRAJ PHARMACEUTICALS LLP,Active,2019-10-19,"Wholesale trade and commission trade, except o...",308/11 DISTT CENTRE JANAKPURI NEW DELHI West D...,RoC-Delhi,managerfinance@smplgroup.org,,2019,LLP,LLP,,,,active
5,AAN-9892,SUDEVI INTELLECT LLP,Active,2019-01-10,Other Business Activities,"SHOP NO.148, BOULEVARD MAJIWADA, LODHA PARADIS...",RoC-Mumbai,raghav.agarwal@sudevichemicals.com,,2019,LLP,LLP,,,,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283293,AAI-5176,DK BIOSCIENCES LLP,Active,2017-02-09,Other Business Activities,Opposite Kalla Petrol Pump NH-15 Jaisalmer Roa...,RoC-Jaipur,dkbiosciences@chura.co.in,334001,2017,LLP,LLP,,,,active
283294,AAN-4195,KROZONE TECHNOLOGIES LLP,Active,2018-10-11,Other Business Activities,"306, ABHISHILP COMPLEX, NR. VISHWESHWAR MAHADE...",RoC-Ahmedabad,JASENT1997@GMAIL.COM,380015,2018,LLP,LLP,,,,active
283295,AAO-5269,WINGS TECHNO PRODUCT LLP,Active,2019-03-13,Other Business Activities,"NO.2, 1ST STREET, GOVINDHARAJAPURAM, NA ADYAR ...",RoC-Chennai,apex@live.in,600020,2019,LLP,LLP,,,,active
283299,AAI-0333,ANSH REAL ESTATES LLP,Active,2016-12-16,Other Business Activities,H NO. 1246 SECTOR 22 B NA CHANDIGARH Chandigar...,RoC-Chandigarh,hkpuri_2007@rediffmail.com,160022,2016,LLP,LLP,,,,active


In [36]:
import pandas as pd

env = "uat"

dbName = "masters"

collection = "assurekitcompanymasters"

q = connectionScript(env = env, dbName = dbName, collectionName= collection)

In [37]:
data = list(q.collection.find({"class":"LLP"},{"_id":0,"companyId":1}))

In [38]:
data = [x['companyId'] for x in data]

In [39]:
len(data)

103294

In [40]:
llpsToBeIncorporated = [x for x in list(df['companyId']) if x not in data]

In [41]:
dfFinal = df[df['companyId'].isin(llpsToBeIncorporated)]

In [42]:
dfFinal['id'] = [x for x in range(1262569, 1262569 +len(dfFinal.index))]

C:\Users\sheld\AppData\Local\Temp/ipykernel_4988/3709122067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFinal['id'] = [x for x in range(1262569, 1262569 +len(dfFinal.index))]


In [43]:
dfFinal['id'] = dfFinal['id'].astype('str')

C:\Users\sheld\AppData\Local\Temp/ipykernel_4988/1763977857.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFinal['id'] = dfFinal['id'].astype('str')


In [44]:
dfFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11819 entries, 237665 to 283295
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   companyId              11819 non-null  object
 1   name                   11819 non-null  object
 2   govStatus              11819 non-null  object
 3   registrationDate       11819 non-null  object
 4   business               11819 non-null  object
 5   address                11819 non-null  object
 6   registrationAuthority  11819 non-null  object
 7   emailAddress           11819 non-null  object
 8   companyPincode         11819 non-null  object
 9   dateOfIncorporation    11819 non-null  int64 
 10  class                  11819 non-null  object
 11  subCategory            11819 non-null  object
 12  registrationState      11819 non-null  object
 13  authorisedCapital      11819 non-null  object
 14  paidUpCapital          11819 non-null  object
 15  status       

In [ ]:
df.to_excel()

In [47]:
dfFinal.to_dict("records")[:1]

[{'companyId': 'AAN-4590',
  'name': 'TAX WITHOUT TEARS LLP',
  'govStatus': 'Active',
  'registrationDate': '2018-10-23',
  'business': 'Other Business Activities',
  'address': 'Block - KD, House No - 209, Pitampura Near Kohat Enclave Metro Station New Delhi North West DL 110088 IN',
  'registrationAuthority': 'RoC-Delhi',
  'emailAddress': ' INFO@SAAWARIYA.IN',
  'companyPincode': '',
  'dateOfIncorporation': 2018,
  'class': 'LLP',
  'subCategory': 'LLP',
  'registrationState': '',
  'authorisedCapital': '',
  'paidUpCapital': '',
  'status': 'active',
  'id': '1262569'}]

In [48]:
dfFinal.to_json("llc" + datetime.today().strftime("%d-%m-%Y") +".json",orient = "records")

In [49]:
dfFinal

,companyId,name,govStatus,registrationDate,business,address,registrationAuthority,emailAddress,companyPincode,dateOfIncorporation,class,subCategory,registrationState,authorisedCapital,paidUpCapital,status,id
237665,AAN-4590,TAX WITHOUT TEARS LLP,Active,2018-10-23,Other Business Activities,"Block - KD, House No - 209, Pitampura Near Koh...",RoC-Delhi,INFO@SAAWARIYA.IN,,2018,LLP,LLP,,,,active,1262569
237714,AAB-4836,R C CHADDA & CO LLP,Active,2013-04-25,Other Business Activities,"A-27, VIVEK VIHAR PHASE-II OPP. SUKHDEV COLLEG...",RoC-Delhi,RCCANDCO@GMAIL.COM,,2013,LLP,LLP,,,,active,1262570
237715,AAA-1684,RELAN REALTY LLP,Active,2010-06-28,Real estate activities,"Unit No. 5, Basement, ""M.G Centrum"" Sultanpur,...",RoC-Delhi,PNRSEC@PNRGROUPINDIA.COM,,2010,LLP,LLP,,,,active,1262571
237719,AAQ-6796,CUTE COINS INDIA LLP,Active,2019-09-27,Other Business Activities,"C/O DHARMENDER SINGH TOLNI VILLAGE , SOHNA GUR...",RoC-Delhi,CS@SETINDIABIZ.COM,,2019,LLP,LLP,,,,active,1262572
237747,AAE-9278,FLARE ELECTRICALS LLP,Active,2015-10-14,Other Business Activities,"BLD-10/503, CHINAR GREENWAYS CHS LTD.AGARWAL C...",RoC-Mumbai,kiran_gujaran06@rediffmail.com,401104,2015,LLP,LLP,,,,active,1262573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283285,AAG-7656,RADHEY GLOBAL LLP,Active,2016-06-24,Manufacture of food products and beverages,"SHOP NO 18, NEAR SANSKRUTIK BHAVAN, MORSHI ROA...",RoC-Mumbai,yrz5751@gmail.com,444603,2016,LLP,LLP,,,,active,1274383
283290,AAD-8722,SAIPRESS PRINTS LLP,Active,2015-05-05,"Publishing, printing and reproduction of recor...","Survey No.36, , Hiss No. 2,4, 36/2/3/7 Near Da...",RoC-Pune,viral.saipress@gmail.com,411048,2015,LLP,LLP,,,,active,1274384
283293,AAI-5176,DK BIOSCIENCES LLP,Active,2017-02-09,Other Business Activities,Opposite Kalla Petrol Pump NH-15 Jaisalmer Roa...,RoC-Jaipur,dkbiosciences@chura.co.in,334001,2017,LLP,LLP,,,,active,1274385
283294,AAN-4195,KROZONE TECHNOLOGIES LLP,Active,2018-10-11,Other Business Activities,"306, ABHISHILP COMPLEX, NR. VISHWESHWAR MAHADE...",RoC-Ahmedabad,JASENT1997@GMAIL.COM,380015,2018,LLP,LLP,,,,active,1274386


In [46]:
#dfFinal.to_excel("llc2023-05-23.xlsx",index = False)